## 1 Introduction

This tutorial introduces the basic features for simulating titratable systems via the constant pH method.
The constant pH method is one of the methods implemented for simulating systems with chemical reactions within the Reaction Ensemble module. It is a Monte-Carlo method designed to model an acid-base ionization reaction at a given (fixed) value of solution pH.

We will consider a homogeneous aqueous solution of a titratable acidic species $\mathrm{HA}$ that can dissociate in a reaction, that is characterized by the equilibrium constant $\mathrm{p}K_A=-\log_{10} K_A$
$$\mathrm{HA} \Leftrightarrow \mathrm{A}^- + \mathrm{H}^+$$


If $N_0 = N_{\mathrm{HA}} + N_{\mathrm{A}^-}$ is the number of titratable groups in solution, then we define the degree of dissociation $\alpha$ as:

$$\alpha = \dfrac{N_{\mathrm{A}^-}}{N_0}.$$

This is one of the key quantities that can be used to describe the acdi-base equilibrium. Usually, the goal of the simulation is to predict the value of $\alpha$ under given conditions in a complex system with intearctions. In this tutorial, we will simulate only ideal systems (without intermolecular interactions) to show that in such case our simulations reproduce the well-known analytical solutions.

### 1.1 The Chemical Equilibrium and Reaction Constant

The equilibrium reaction constant describes the chemical equilibrium of a given reaction. The values of equilibrium constants for various reactions can be found in tables. For the acid-base ionization reaction, the equilibrium constant is conventionally called the acidity constant, and it is defined as
\begin{equation}
K_A = \frac{a_{\mathrm{H}^+} a_{\mathrm{A}^-} } {a_{\mathrm{HA}}}
\end{equation}
where $a_i$ is the activity of species $i$. It is related to the chemical potential and to the concentration
\begin{equation}
\mu_i = k_{\mathrm{B}}T \ln a_i
\,,\qquad
a_i = \frac{c_i \gamma_i}{c^{\ominus}}
\end{equation}
where $\gamma_i$ is the activity coefficient, and $c^{0}$ is the (arbitrary) reference cocnentration, often chosen to be the standard concentration, $c^0 = 1\,\mathrm{mol\,dm^{-3}}$.
Note that $K$ is a dimensionless quantity but its numerical value depends on the choice of $c^0$.
For an ideal system, $\gamma_i=0$ by definition, whereas for an interacting system $\gamma_i$ is a non-trivial function of the interactions. Therefore, for an ideal system we can rewrite $K$ in terms of equilibrium concentrations
\begin{equation}
K_A \overset{\mathrm{ideal}}{=} \frac{c_{\mathrm{H}^+} c_{\mathrm{A}^-} } {c_{\mathrm{HA}} c^{\ominus}}
\end{equation}

The ionization degree can also be expressed via the ratio of concentrations:
\begin{equation}
\alpha 
= \frac{N_{\mathrm{A}^-}}{N_0} 
= \frac{N_{\mathrm{A}^-}}{N_{\mathrm{HA}} + N_{\mathrm{A}^-}}
= \frac{c_{\mathrm{A}^-}}{c_{\mathrm{HA}}+c_{\mathrm{A}^-}}
= \frac{c_{\mathrm{A}^-}}{c_{0}}.
\end{equation}
where $c_0=c_{\mathrm{HA}}+c_{\mathrm{A}^-}$ is the total concentration of titratable groups irrespective of their ionization state.
Then, we can characterize the acid-base ionization equilibrium using the iniozation degree and pH, defined as
\begin{equation}
\mathrm{pH} = -\log_{10} a_{\mathrm{H^{+}}} \overset{\mathrm{ideal}}{=} -\log_{10} (c_{\mathrm{H^{+}}} / c^{\ominus})
\end{equation}
Substituting for the ionization degree and pH into the expression for $K_A$ we obtain the Henderson-Hasselbalch equation
\begin{equation}
\mathrm{pH}-\mathrm{p}K_A = \log_{10} \frac{\alpha}{1-\alpha}
\end{equation}
One important implication of the Henderson-Hasselbalch equation is that at a fixed pH value the ionization degree of an ideal acid is independent of concentration. Another important implication is that it does not depend on the absolute values of $\mathrm{p}K_A$ or $\mathrm{pH}$, but only on the difference, $\mathrm{pH}-\mathrm{p}K_A$.

### 1.2 Constant pH Method

The constant pH method is designed to simulate an acid-base ionization reaction at a given pH. It assumes that the simulated system is coupled to an implicit reservoir of $\mathrm{H^+}$ ions but 
exchange of ions with this reservoir is not explicitly simulated. Therefore, the number of $\mathrm{H}^+$ ions in the simulation box does not correspond to the chosen pH. This may lead to artifacts when simulating interacting systems, especially at high of low pH values. Discussion of these artifacts is beyond the scope of this tutorial (see e.g. Ref.[1] for further details).

In Espresso, the forward step of the ionization reaction (from left to right) is implemented by 
changing the chemical identity (particle type) of a randomly selected $\mathrm{HA}$ particle to $\mathrm{A}^-$, and inserting another particle that represents $\mathrm{H}^+$. In the reverse direction (from right to left), the chemical identity (particle type) of a randomly selected $\mathrm{A}^{-}$ is changed to $\mathrm{HA}$, and a randomly selected $\mathrm{H}^+$ is deleted from the simulation box. The probability of proposing the  forward reaction step is $P_\text{prop}=N_\mathrm{HA}/N_0$, and probability of proposing the reseverse step is $P_\text{prop}=N_\mathrm{A}/N_0$  [1]. The trial move is accepted with the acceptance probability

$$ P_{\mathrm{acc}} = \operatorname{min}\left(1, \exp(\beta \Delta E_\mathrm{pot} \pm \ln_{10} (\mathrm{pH - p}K_A) ) \right)$$

Here $\Delta E_\text{pot}$ is the potential energy change due to the reaction, while $\text{pH - p}K$ is an input parameter. 
The signs $\pm 1$ correspond to the forward and reverse direction of the ionization reaction, respectively. 



## 2 Setup
We start by creating a system instance with an arbitrary box length of 35 $\sigma$ and creating `N0` titratable units (in the associated state). We set the dissociation constant of the acid to $\mathrm{p}K_A=4.25$, that is the acidity constant of acrylic acid.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.ion()

import espressomd
from espressomd import reaction_ensemble

# System parameters
#############################################################
BOX_L= 35 # box size is arbitrary (see Henderson-Hasselbalch equation)

# particle types of different species
TYPE_HA = 0
TYPE_A = 1
TYPE_H = 2

# acidity constant
pK = 4.25
K = 10**(-pK)

# range of acidity constants to be used in calculations
pKmin=pK-3
pKmax=pK+3

N0 = 10  # number of titratable units in the box

# Integration parameters
#############################################################
system = espressomd.System(box_l=[BOX_L] * 3)
system.set_random_state_PRNG()
np.random.seed(seed=10)

for i in range(N0):
    system.part.add(pos=np.random.random(3) * system.box_l, type=TYPE_HA)

In the first step, we initialize the reaction ensemble, by setting the temperature, exclusion radius and seed of the random number generator. Because we are simulating an ideal system, 
we set the temperature and exclusion radius to arbitrary values. In an interacting system the exclusion radius ensures that particle insertions too close to other particles are not attempted. Such insertions would make the subsequent Langevin dynamics integration unstable. We set the seed to a constant value to ensure reproducible sequence of random numbers when repeating the exercise. For production runs, one would set the 

The next step is to define the reaction system and the seed of the pseudo-random number generator which is used for the Monte-Carlo steps.

The order in which species are written in the lists of reactants and products is very important. When a reaction move is performed, identity of the first species in the list of reactants is changed to the first species in the list of products, the second reactant species is changed to the second product species, and so on. If the reactant list has more species than the product list, then excess reactant species are deleted from the system. If the product list has more species than the reactant list, then product the excess product species are created and randomly placed inside the simulation box. This is especially relevant if some of the species belong to a cahin-like molecule.

In the example below, the order of reactants and produces ensures that identity of HA is changed to $\mathrm{A^{-}$ and vice versa, while $\mathrm{H^{+}}$ is inserted/deleted in the reaction move. Reversing the order of products in our reaction (i.e. from  `product_types=[TYPE_H, TYPE_A]` to `product_types=[TYPE_A, TYPE_H]`), would result in a reaction move, where the identity HA would be changed to $\mathrm{H^{+}}$, while $\mathrm{A^{-}}$ would be inserted/deleted at a random position in the box.


We also assign charges to each type because in general the charge will play a role in simulations with electrostatic interactions. As an easy task for the interested reader we propose to adapt the tutorial to account for electrostatic interactions. Therefore we keep these values for the needed charge assignments in place, although they are not needed for an ideal system.

In [ ]:
RE = reaction_ensemble.ConstantpHEnsemble(
        temperature=1, exclusion_radius=0.0, seed=77)

RE.add_reaction(gamma=K, reactant_types=[TYPE_HA], reactant_coefficients=[1],
                product_types=[TYPE_A, TYPE_H], product_coefficients=[1, 1],
                default_charges={TYPE_HA: 0, TYPE_A: -1, TYPE_H: +1})
print(RE.get_status())

Next we perform simulations at different pH values. The system must be equilibrated at each pH before taking samples.
Calling `RE.reaction(X)` attempts in total `X` reactions (in back and forward directions).
We also plot the acceptance rate for the dissociation reaction and the association reaction for the first pH value which we set.

In [ ]:
num_samples = 100

pHs = np.linspace(pKmin, pKmax, num=15)
degrees_of_dissociation = []
std_dev_degree_of_dissociation = []
histograms = []
histogram_edges = range(0,N0+1)

for pH in pHs:
    print("pH is {:.1f}".format(pH))
    RE.constant_pH = pH
    RE.reaction(4 * N0) # equilibration to the new pH
    numAs = []
    for i in range(num_samples):
        RE.reaction(N0 + 1)
        numAs.append(system.number_of_particles(type=TYPE_A))
    degrees_of_dissociation.append(np.mean(numAs) / N0)
    std_dev_degree_of_dissociation.append(np.std(numAs, ddof=1) / N0)
    print("occurred particle numbers of type A ", sorted(set(numAs)))
    histogram = np.bincount(np.array(numAs).astype(int), minlength=N0 + 1)
    histograms.append(histogram / float(len(numAs)))
    if(abs(pH - 1.0) < 1e-4):
        print("acceptance rate dissociation", RE.get_acceptance_rate_reaction(0))
        print("acceptance rate association", RE.get_acceptance_rate_reaction(1))

## 3 Results
Finally we plot our results and compare it to the well-known result for an ideal reacting system $\alpha(\mathrm{pH}, \mathrm{p}K)$ presented above.
### 3.1 Statistical Uncertainty

To quantify the simulation uncertainty, we will add error bars to the calculated data points.
Since the underlying probability distribution of our data does not follow a well-known function, such as the Gaussian,
Binomial, exponential or Poisson distributions, there is no literature to provide us with an analytical form of the
confidence interval. Sometimes the analytical form of the underlying probability distribution is also unknown.
However, a _statistic_ calculated from a random sample often has a probability distribution different from the
sample probability distribution. We will use that to our advantage.

In our case, the values of $\alpha$ are drawn from a random distribution $X(N_{\mathrm{A}}, N_{\mathrm{0}})$ with _parameters_ $N_{\mathrm{A}}$, $N_{\mathrm{0}}$ and moments $\mu_\alpha = \operatorname{E}(X)$ and $\sigma_\alpha = E([E(X)-X]^2)$.
The average $\bar{\alpha} = \frac{1}{n}\sum_{i=1}^n \alpha_i$ and variance $\sigma_n^2 = \frac{1}{n}\sum_{i=1}^n (\alpha_i - \bar{\alpha})^2$ of a sample of $n$ independent values $\alpha_i$ are themselves random variables,
but they do not have the same probability distribution as $X$. They are called _statistics_ because they are calculated
from a sample of the population, and can be used as _estimators_ of the moments $\mu_\alpha$ and $\sigma_\alpha$.

According to the Central Limit Theorem (CLT), the random variable $\sqrt{n}(\bar{\alpha} - \mu_\alpha)$ has a probability distribution which converges for large sample sizes $n$ to the probability distribution $\mathcal{N}(0, \sigma_n^2)$. This statement can be rewritten as:

\begin{equation}
    z = \frac{\bar{\alpha} - \mu_\alpha}{\sigma_n} \sqrt{n} \sim \mathcal{N}(0,1)
\end{equation}

in the limit of a large $n$, with $z$ the two-tail $z$-score of the standard Normal distribution.
The rate of convergence depends on the underlying probability distribution $X$. For this tutorial,
15'000'000 data points were collected and the distribution of $z$ was very close to Normal for $n=100$.

The 95% probability of finding the population mean $\mu$ within two boundaries $w^+$ and $w^-$ _over repeated experiments_ is given by:

\begin{equation}
    \operatorname{Pr}\left( w^- \leq \mu \leq w^+\right) = 0.95
\end{equation}

For the standard Normal distribution, by definition $\mu = 0$ and $w^\pm = \pm z_{1-0.95/2}$ with $z_{1-0.95/2} \approx 1.96$, so for the distribution $\sqrt{n}(\bar{\alpha} - \mu_\alpha) / \sigma_n$:

\begin{equation}
    w^\pm = \bar{\alpha} \pm z_{1-0.95/2} \cdot \frac{\sigma_n}{\sqrt{n}}
\end{equation}

To plot the simulated curve with error bars, we need to estimate three parameters: $\mu_\alpha$ and $w^\pm$.
The accuracy of our estimation will depend on the sample size `num_samples`. The larger it is, the more
$\sqrt{n}(\bar{\alpha} - \mu_\alpha)$ converges to $\mathcal{N}(0, \sigma_n^2)$ and the smaller the $w^\pm$ interval gets.

The sample mean $\bar{\alpha}$ is already an unbiased estimator of $\mu_\alpha$.
However to estimate $w^\pm$ we need to estimate $\sigma_N$, for which the sample standard deviation $s_n$ is a biased estimator. We can account for some of the bias using Bessel's correction:

\begin{equation}
    s_{n-1}^2 = \frac{1}{n-1}\sum_{i=1}^n (\alpha_i - \bar{\alpha})^2
\end{equation}

This correction is required because the sample variance $s_n^2$ is evaluated against $\bar{\alpha}$ instead of the
population mean $\mu_\alpha$. $s_{n-1}^2$ is an unbiased estimator of $\sigma_n^2$, however taking the square root
introduces a new source of bias due to Jensen's inequality. Different approaches exist to correct for this bias,
but since it is small, it is common to neglect it. The quantity $w^\pm$ is estimated by a statistic called the 95% Confidence Interval:

\begin{equation}
    \mathrm{CI}_{95\%} = \bar{\alpha} \pm z_{1-0.95/2} \cdot \frac{s_{n-1}}{\sqrt{n}}
\end{equation}

We will use that estimate to plot the error bars.

In [ ]:
import scipy.stats
def ideal_degree_of_dissociation(pH, pK):
    return 1. / (1 + 10**(pK - pH))

z = scipy.stats.norm.interval(1 - 0.05 / 2, loc=0)[1] # two-tail z-score at 95%
ci95 = z * np.array(std_dev_degree_of_dissociation) / np.sqrt(num_samples)
pK = -np.log10(K)
plt.figure(figsize=(10, 6), dpi=80)
plt.errorbar(pHs - pK, degrees_of_dissociation, ci95, label=r"simulation")
pHs2 = np.linspace(pKmin, pKmax, num=50)
plt.plot(pHs2 - pK, ideal_degree_of_dissociation(pHs2, pK), label=r"ideal")
plt.xlabel('$pH-pK$', fontsize=20)
plt.ylabel(r'$\alpha$', fontsize=20)
plt.legend(fontsize=20)
plt.show()

We now compare the relative frequencies $h$ of observed numbers of particles of type A to the probabilities $p$ which are expected in the constant pH ensemble for a non-interacting system [1]:

\begin{equation}
    p(N_\mathrm{A})=\frac{{N_0 \choose N_\mathrm{A}} 10^{(\mathrm{pH-p}K)N_\mathrm{A}}}
        {\sum_{N=0}^{N_0} {N_0 \choose N} 10^{(\mathrm{pH}-\mathrm{p}K)N}}
\end{equation}

In [ ]:
import scipy.special
def constant_pH_numA_pmf(numA, pH, pK, N0):
        all_NAs = range(N0+1)
        return scipy.special.binom(N0, numA) * 10.0**((pH - pK) * numA) / \
            sum(scipy.special.binom(N0, i) * 10**((pH - pK) * i) for i in all_NAs)

NAs = range(N0 + 1)

from IPython import display
import time

for index_pH in range(pHs.shape[0]):
    fig, ax = plt.subplots(1, 1)
    pH = pHs[index_pH]
    ax.plot(NAs, constant_pH_numA_pmf(NAs, pH, pK, N0), 'ro', ms=8, mec='r', label="Theory pH "+str(pH))
    ax.vlines(NAs, 0, constant_pH_numA_pmf(NAs, pH, pK, N0), colors='r', lw=4)
    ax.plot(histogram_edges, histograms[index_pH], 'bo', ms=8, mec='b',
            label="Simulation pH " + str(pH))
    ax.vlines(histogram_edges, 0, histograms[index_pH], colors='b', lw=1)
    ax.legend()
    ax.set_xlabel('$N_\\mathrm{A}}$')
    ax.set_ylabel('$p(N_\mathrm{A})$ or $h(N_\mathrm{A})$')
    display.clear_output(wait=True)
    display.display(plt.gcf())
    time.sleep(2.0)

## References

[1] Landsgesell, Jonas, Christian Holm, and Jens Smiatek. Simulation of weak polyelectrolytes: a comparison between the constant pH and the reaction ensemble method. The European Physical Journal Special Topics 226.4 (2017): 725-736.